# eXplainable A.I.


## Dataset: Tokyo Fish Market

Stimare con la maggiore precisione possibile il peso dei pesci. La misura utilizzata per l'errore è **RMSE**.

<br>

![tfm](https://www.driveontheleft.com/wp-content/uploads/2017/10/Fish-market-22-min.jpg)

<br>

* 294 samples
* 7 features compreso il target

* Length1: standard length
* Length2: fork length
* Lenght3: total length
* Height
* Width
* Species: 7 categorie
* ***Weight*** (target)

<br>

![fish](https://www.fishbase.de/Images/Glospic/G_Fig13a6181_SL.jpg)

<br>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
fish= pd.read_csv("./dati/dati.csv")

## Prima analisi esplorativa

In [ ]:
fish.shape

In [ ]:
fish.head()

In [ ]:
fish.describe()

Ci sono campioni con peso 0, eliminiamo

In [ ]:
# controllo che le features siano scritte correttamente (spazi, ...)
fish.columns

In [ ]:
fish = fish.loc[fish.Weight!=0,:].copy()

In [ ]:
# controllo missing
fish.isna().sum()

**AGGIORNAMENTO:** Le variabili Length sono collineari, tengo solo la L3, la coda può avere un peso nella regressione di casi particolari (non sono esperto di pesci). Lascio nella nota la prima versione del commento.

> Le variabili Length sono collineari, tengo solo la L2 perchè dovrebbe essere la mediana. Se tenessi tutte e tre avrei ottimi risultati in train - validation ma pessimi in test. **Overfitting**.

In [ ]:
fish.drop(["Length1", "Length2"], axis=1, inplace=True)

## Analisi visuale

In [ ]:
plt.figure(figsize=(18,10))
for s in fish.Species.unique():
    l = fish.loc[fish.Species==s, "Length3"]
    h = fish.loc[fish.Species==s, "Height"]
    plt.scatter(x=l, y=h, marker="o")
plt.legend(fish.Species.unique())
plt.show()

In [ ]:
plt.figure(figsize=(18,10))
for s in fish.Species.unique():
    l = fish.loc[fish.Species==s, "Length3"]
    h = fish.loc[fish.Species==s, "Width"]
    plt.scatter(x=l, y=h, marker="o")
plt.legend(fish.Species.unique())
plt.show()

In [ ]:
plt.figure(figsize=(18,10))
for s in fish.Species.unique():
    l = fish.loc[fish.Species==s, "Width"]
    h = fish.loc[fish.Species==s, "Height"]
    plt.scatter(x=l, y=h, marker="o")
plt.legend(fish.Species.unique())
plt.show()

Chiaramente le tre variabili numeriche sono correlate, la specie è una discriminante.

Conto le occorrenze per specie, sono distribuite abbastanza uniformemente nel dataset.

In [ ]:
fish.Species.value_counts()

In [ ]:
plt.figure(figsize=(18,10))
fish.Species.hist()
plt.show()

## Train - Validation split

Uso il campionamento stratificato sulla specie.

In [ ]:
X = fish.drop(["Weight"], axis=1).copy()
y= fish["Weight"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=X.Species)

## Preprocessing
Per i valori numerici uso *RobustScaler*, basato sulla distanza interquartile perchè più robusto ad eventuali valori anomali.

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
scaler = RobustScaler()
numeric_df = scaler.fit_transform(X_train.iloc[:,:3])
numeric_val = scaler.transform(X_val.iloc[:,:3])

In [ ]:
encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
cat_df = encoder.fit_transform(X_train[["Species"]])
cat_val = encoder.transform(X_val[["Species"]])

Visto che sicuramente le tre componenti numeriche sono correlate voglio vedere in che relazione sono con il peso. Applico una PCA e tengo solo la prima componente principale per fare lo scatterplot. Potrebbe essere rinominata *Stazza* perchè riassume le tre dimensioni.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
x_pca = pca.fit_transform(numeric_df)

In [ ]:
plt.figure(figsize=(18,10))
plt.scatter(x_pca, y_train)
plt.show()

Il plot mi aiuta a scegliere il grado del polinomio della mia regressione, come si vede la relazione non è lineare ma di grado 2. Potrei usara un regressione ad alberi ma preferisco utilizzare un modello parametrico visto che i modelli ad albero restituiscono la media dei valori nel nodo avrei una funzione *a gradini* mentre qui la relazione è abbastanza visibile.

## Interpretable A. I.

I modelli glassbox sono progettati per essere interpretabili, ne sono un esempio i modelli parametrici (come la regressione lineare, polinomiale e logistica dove è possibile interpretare i beta), i GAMs, le regole associative e il KNN.
In questo notebook vedremo alcuni di questi esempi e poi passeremo a qualche esempio di eXplainable A.I. con la *Permutation Importance* e ***SHAP***.

<br>

![galssbox](https://www.forniturealberghiereonline.it/images/cucina-a-vista-ristorante-con-sgabelli.jpg)

<br>

### Linear Model

Il primo esempio riguarda la regressione lineare, è il modello più semplice in statistica e ML ma tanti dei modelli più usati ne sono una sua estensione.
La versione più semplice della regressione lineare è predirre una variabile quantitativa continua *Y* in base ai valori della variabile indipendente *X*, il modello sarà $Y \approx \beta_0 + \beta_1 X$.

La regressione lineare non fa altro che calcolare un valore stimato dei coefficienti β per trovare una retta che minimizzi l'errore $\hat{y} = \hat{\beta_0} + \hat{\beta_1}x$.

#### Wrong Model: Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
X_train_lr = np.hstack((numeric_df, cat_df))
X_val_lr = np.hstack((numeric_val, cat_val))

In [ ]:
lr = LinearRegression()
lr.fit(X_train_lr, y_train)
print("R2 on train: ", lr.score(X_train_lr, y_train))
print("R2 on validation: ", lr.score(X_val_lr, y_val))

Vediamo i coefficienti, $\beta_0$ è l'intercetta, questo è il valore che avrà la nostra previsione $\hat{y}$ se tutti i regressori sono pari a 0.

In [ ]:
lr.intercept_

I coefficienti di $\beta_i$ rappresentano l'incremento (o il decremento) del peso del pesce per ogni unità dei nostri regressori. Ad esempio per ogni unità in più in lunghezza il peso del pesce sale di 643.68.

In [ ]:
lr.coef_

In [ ]:
yhat_lr=lr.predict(X_val_lr)
print("MSE on Validation: ", mean_squared_error(y_val, yhat_lr))
print("RMSE on Validation: ", np.sqrt(mean_squared_error(y_val, yhat_lr)))

In [ ]:
yhat_train = lr.predict(X_train_lr)
plt.figure(figsize=(18,10))
plt.scatter(x_pca, y_train)
plt.scatter(x_pca, yhat_train)
plt.legend(["yreal", "yhat"])
plt.title("Grafico Previsione")
plt.show()

Dal grafico vediamo chiaramente che non possiamo usare la regressione lineare per variabili che hanno un rapporto non lineare, costruiamo quindi la nostra baseline usando una regressione polinomiale.

#### Baseline Model: Polynomial Regression

Per utilizzare la regressione polinomiale in scikit-learn dobbiamo costruire le features come la combinazione di tutti i polinomi minori e uguali al grado scelto. Nel nostro caso il grado del polinomio è pari a 2.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)


In [ ]:
poly_train=poly.fit_transform(numeric_df)
poly_val = poly.fit_transform(numeric_val)

In [ ]:
X_train_new = np.hstack((poly_train, cat_df))
X_val_new = np.hstack((poly_val, cat_val))

In [ ]:
PolyR = LinearRegression()

In [ ]:
PolyR.fit(X_train_new, y_train)
print("R2 on train: ", PolyR.score(X_train_new, y_train))
print("R2 on validation: ", PolyR.score(X_val_new, y_val))

In [ ]:
yhat=PolyR.predict(X_val_new)

In [ ]:
print("MSE on Validation: ", mean_squared_error(y_val, yhat))
print("RMSE on Validation: ", np.sqrt(mean_squared_error(y_val, yhat)))

In [ ]:
print("Intercetta: ", PolyR.intercept_, "\n")
print("Coefficienti: ", PolyR.coef_)

In [ ]:
yhat_train = PolyR.predict(X_train_new)
plt.figure(figsize=(18,10))
plt.scatter(x_pca, y_train)
plt.scatter(x_pca, yhat_train)
plt.legend(["yreal", "yhat"])
plt.show()

### Regularization

<br>

![regularization](https://miro.medium.com/max/1400/1*nrWncnoJ4V_BkzEf1pd4MA.png)

<br>

La regolarizzazione è un metodo che inserisce un parametro nella calcolo della funzione di costo (i minimi quadrati) in modo da *"ridurre"* (shrink) la dimensione dei coefficienti e portarli verso lo zero e abbassare la varianza. Questo metodo migliora molto la prestazione dei modelli (non solo lineari).

#### Ridge Regression

La ridge regression usa la regolarizzaizone $L_2$, il paramtero aggiuntivo è $\alpha \sum_{i=1}^n \beta_i ^2$. α è il parametro che regola l'effetto della regolarizzazione. Con questo metodo oltre a non esserci grosse differenze tra i coefficienti i meno importanti saranno molto vicini allo 0. Per saperne di più potete andare [qui](https://andreaprovino.it/ridge-regression/).

Uso la versione *Cross-Validation* disponibile su scikit-learn per valutare i diversi valori di *alpha*.

*N.B. Scegliere il giusto valore di $\alpha$ non è semplice e può portare a modelli eccessivamente sparsi.*

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import RidgeCV

In [ ]:
PolyRCV = RidgeCV(cv=10)
cv_score = cross_validate(PolyRCV, X_train_new, y_train, cv = 10, n_jobs=-1, return_estimator=True, return_train_score=True)

In [ ]:
print(cv_score['train_score'].mean(), "+-", cv_score['train_score'].std())
print(cv_score['test_score'].mean(), "+-", cv_score['test_score'].std())

In [ ]:
PolyRCV.fit(X_train_new, y_train)
print("Parametro alpha per la regolarizzazione: ", PolyRCV.alpha_)
print("R2 on train: ", PolyRCV.score(X_train_new, y_train))
print("R2 on validation: ", PolyRCV.score(X_val_new, y_val))

In [ ]:
yhatCV=PolyRCV.predict(X_val_new)
print("MSE on Validation: ", mean_squared_error(y_val, yhatCV))
print("RMSE on Validation: ", np.sqrt(mean_squared_error(y_val, yhatCV)))

In [ ]:
print("Intercetta: ", PolyRCV.intercept_, "\n")
print("Coefficienti: ", PolyRCV.coef_)

In [ ]:
yhat_trainCV = PolyRCV.predict(X_train_new)
plt.figure(figsize=(18,10))
plt.scatter(x_pca, y_train)
plt.scatter(x_pca, yhat_trainCV)
plt.legend(["yreal", "yhat"])
plt.title("Ridge Regression")
plt.show()

#### Lasso Regression

Se abbiamo tante variabili, c'è multicollinearità, dobbiamo semplificare il modello, ... una scelta può essere la Lasso Regression.
Questo modello applica una regolarizzaizone $L_1$ portando a zero i coefficienti delle variabili meno impattanti. Il parametro di regolarizzaizone è $\alpha \sum_{i=1}^n |\beta_i|$.

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
PolyLCV = LassoCV(cv=10)
cv_score = cross_validate(PolyLCV, X_train_new, y_train, cv = 10, n_jobs=-1, return_estimator=True, return_train_score=True)

In [ ]:
print(cv_score['train_score'].mean(), "+-", cv_score['train_score'].std())
print(cv_score['test_score'].mean(), "+-", cv_score['test_score'].std())

In [ ]:
PolyLCV.fit(X_train_new, y_train)
print("Parametro alpha per la regolarizzazione: ", PolyLCV.alpha_)
print("R2 on train: ", PolyLCV.score(X_train_new, y_train))
print("R2 on validation: ", PolyLCV.score(X_val_new, y_val))

In [ ]:
yhatLCV=PolyLCV.predict(X_val_new)
print("MSE on Validation: ", mean_squared_error(y_val, yhatLCV))
print("RMSE on Validation: ", np.sqrt(mean_squared_error(y_val, yhatLCV)))

In [ ]:
print("Intercetta: ", PolyLCV.intercept_, "\n")
print("Coefficienti: ", PolyLCV.coef_)

In [ ]:
yhat_trainCV = PolyLCV.predict(X_train_new)
plt.figure(figsize=(18,10))
plt.scatter(x_pca, y_train)
plt.scatter(x_pca, yhat_trainCV)
plt.legend(["yreal", "yhat"])
plt.title("Lasso Regression")
plt.show()

### Confronto tra i modelli
#### Analisi Visuale

In [ ]:
columns = poly.get_feature_names() + encoder.categories_[0].tolist()

In [ ]:
plt.figure(figsize=(18,10))
plt.bar(columns, PolyR.coef_)
plt.bar(columns, PolyRCV.coef_, align='edge', color = "orange")
plt.bar(columns, PolyLCV.coef_, align='edge', color="green")
plt.legend(["PolyRegression", "Ridge", "Lasso"])
plt.title("Polynomial Regression vs. Ridge vs. Lasso.\n Differenze tra i coefficienti")
plt.show()

In [ ]:
plt.figure(figsize=(18,10))
plt.bar(columns, PolyR.coef_)
plt.bar(columns, PolyRCV.coef_, align='edge', color = "orange")
plt.legend(["PolyRegression", "Ridge"])
plt.title("Polynomial Regression vs. Ridge.\n Differenze tra i coefficienti")
plt.show()

In [ ]:
plt.figure(figsize=(18,10))
plt.bar(columns, PolyR.coef_)
plt.bar(columns, PolyLCV.coef_, align='edge', color="green")
plt.legend(["PolyRegression", "Lasso"])
plt.title("Polynomial Regression vs. Lasso.\n Differenze tra i coefficienti")
plt.show()

In [ ]:
plt.figure(figsize=(18,10))
plt.bar(columns, PolyRCV.coef_, color ="orange")
plt.bar(columns, PolyLCV.coef_, align='edge', color="green")
plt.legend(["Ridge", "Lasso"])
plt.title("Ridge vs. Lasso.\n Differenze tra i coefficienti")
plt.show()

##### Conclusioni
Come abbiamo potuto vedere i modelli parametrici di cui i lineari ne sono l'esempio più famoso sono facili da interpretare anche per uno specialista di dominio. Altri modelli facilmente interpretabili sono il KNN e i Decision Tree (diventa più complesso quando l'albero fa da base a modelli di bagging o boosting), in generale i modelli basati su regole.

## eXplainable A.I.

L'AI spiegabile (eXplainable Artificial Intelligence, XAI) è un insieme di metodi e processi che consentono agli utenti di comprendere e considerare attendibili i risultati e l'output creati dagli algoritmi di machine learning. L'AI spiegabile viene utilizzata per descrivere un modello AI, il relativo impatto previsto ed i potenziali errori. Aiuta a caratterizzare la precisione, la correttezza, la trasparenza e i risultati del modello nel processo decisionale con tecnologia AI. L'AI spiegabile è fondamentale per un'organizzazione nello sviluppo della fiducia e della sicurezza quando vengono messi in produzione i modelli AI. Inoltre, l'AI spiegabile aiuta le organizzazioni ad adottare un approccio responsabile allo sviluppo AI.

#### Support Vector Machine

Le support vector machine sono dei modelli di apprendimento supervisionato associati ad algoritmi di apprendimento per la regressione e la classificazione. Un modello SVM è una rappresentazione degli esempi come punti nello spazio, mappati in modo tale che gli esempi appartenenti alle due diverse categorie siano chiaramente separati da uno spazio il più possibile ampio.

<br>

![svmr](https://miro.medium.com/max/1750/1*nrXHNqC_hqpyux7GUbtqAQ.png)

<br>

Nella regressione le SVM ci danno la flessibilità di scegliere quanto l'errore è accettabile e trovare un hyperpiano che rappresenti i dati. Per fare ciò la SVM si avvale della funzione kernel.

*I metodi kernel si approcciano al problema mappando i dati in uno spazio di caratteristiche multidimensionale, dove ogni coordinata corrisponde a una caratteristica dei dati dell'elemento, trasformando i dati in un insieme di punti dello spazio euclideo. 
Poiché la mappatura può essere generale (per esempio, non necessariamente lineare), le relazioni trovate in questo modo sono di conseguenza molto generali. I metodi kernel si chiamano così per le funzioni kernel, che vengono usate per operare nello spazio delle caratteristiche senza calcolare le coordinate dei dati nello spazio, ma piuttosto calcolando il prodotto interno tra le immagini di tutte le coppie di dati nello spazio funzione.*

Nel nostro caso useremo un kernel polinomiale di grado 2:
* linear: $K(x,y) = x^Ty + c$
* polynomial: $K(x,y) = (x^Ty + c)^d$
* rbf: $K(x,y) = e^- \frac{||x-y||^2}{2σ^2}$

<br>

![kernel](https://miro.medium.com/max/1400/1*mCwnu5kXot6buL7jeIafqQ.png)

<br>

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [ ]:
svr = GridSearchCV(SVR(kernel='poly',degree=2),
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                            "coef0": [0.0,1.0 ,1.5 ,1.8,2.0 ,2.5, 3]
                               })

In [ ]:
svr.fit(X_train_new,y_train)
print(svr.best_estimator_)
svr.best_score_

In [ ]:
print("R2 on train: ", svr.score(X_train_new, y_train))
print("R2 on validation: ", svr.score(X_val_new, y_val))

In [ ]:
yhatSVR=svr.predict(X_val_new)
print("MSE on Validation: ", mean_squared_error(y_val, yhatSVR))
print("RMSE on Validation: ", np.sqrt(mean_squared_error(y_val, yhatSVR)))

In [ ]:
yhat_trainCV = svr.predict(X_train_new)
plt.figure(figsize=(18,10))
plt.scatter(x_pca, y_train)
plt.scatter(x_pca, yhat_trainCV)
plt.legend(["yreal", "yhat"])
plt.title("SVM Regression")
plt.show()

I punti sono molto vicini ma non abbiamo spiegabilità per quanto riguarda l'impatto delle variabili.

### Explanable Machine Learning: eli5 & SHAP

In [ ]:
!pip install shap
!pip install eli5

### Permutation Importance
Permutare i valori delle features ci permette di valutarne l'impatto sul modello, in maniera semplicistica:
* se le performance si abbassano --> Feature importante
* se le performance restano invariate --> Feature senza impatto
* se una permutazione casuale dei valori migliora il modello --> impatto negativo

eli5 produce un plot a semaforo, sumando dal verde al rosso, buona feature - cattiva feature.

Verranno valutati:
* Regressione Polinomiale
* Ridge Regression Polinomiale

Applico prima il modello al train per vedere cosa ha imparato e come, e successivamente alla validation per vedere se le features che reputa importanti generalizzano bene.

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
permPolyR = PermutationImportance(PolyR, random_state=42, cv="prefit").fit(X_train_new, y_train)
eli5.show_weights(permPolyR, feature_names= columns)

In [ ]:
permPolyR = PermutationImportance(PolyR, random_state=42, cv="prefit").fit(X_val_new, y_val)
eli5.show_weights(permPolyR, feature_names= columns)

In [ ]:
permPolyRCV = PermutationImportance(PolyRCV, random_state=42, cv="prefit").fit(X_train_new, y_train)
eli5.show_weights(permPolyRCV, feature_names= columns)

In [ ]:
permPolyRCV = PermutationImportance(PolyRCV, random_state=42, cv="prefit").fit(X_val_new, y_val)
eli5.show_weights(permPolyRCV, feature_names= columns)

### SHAP Plot
SHAP da Shapely Values, un approccio molto usato nella teoria dei giochi, permette di spiegare come impattano le features sul modello. Ha anche dei plot molto belli a mio avviso.

In [ ]:
import shap
shap.initjs()

Creo dei dataframe per vedere il nome delle colonne

In [ ]:
df_train = pd.DataFrame(X_train_new, columns=columns)
df_val = pd.DataFrame(X_val_new, columns=columns)

### Local Explaination

In [ ]:

ex0 = shap.KernelExplainer(svr.predict, df_train)
shap_values0 = ex0.shap_values(df_val)
shap.force_plot(ex0.expected_value, shap_values0[0], df_val.iloc[0,:])

Il local explainer mostra come si è arrivati da il base value alla previsione in output. Ci sono le features per la singola osservazione.

Il loro impatto è dato dal colore:
* "postivo" rosso
* "negativo" blu

e dalla dimensione, più è grande la sezione sulla barra maggiore è lo SHAP value e quindi l'impatto sulla previsione.

Un grafico più chiaro sulla singola istanza è il waterfall plot, questo grafico và letto dal basso verso l'alto, la logica è uguale al precedente plot ma la leggibilità è migliore e possiamo scegliere quante features visualizzare (default: 10)

In [ ]:
# Ridge Regression
explainer = shap.Explainer(PolyRCV, df_train)
shap_values = explainer(df_train)

In [ ]:
print(shap_values[0])
print(shap_values[0].base_values)
print(type(shap_values.base_values[0]))

In [ ]:
shap.waterfall_plot(shap_values[0])

Nonostante abbiamo detto che SHAP è un modello XAI "locale" possiamo spiegare come funziona globalmente con un plot riassuntivo.

* la colonna a sinistra riporta le features
* sull'asse orizzontale abbiamo lo SHAP values e quindi l'impatto sul modello
* il colore indica il valore della variabile

Se abbiamo una distribuzione con la maggior parte dei punti sullo zero questa variabile non ha impatto sul modello indipendentemente dal valore che assume.
Nel caso si *x1* e *x2* valori alti hanno impatto positivo mentre quelli bassi negativo, se guardiamo a *x2^2* è il contrario.

In [ ]:
shap.summary_plot(shap_values0, df_val)